# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [14]:
using Manifolds
using Manopt
using LinearAlgebra
using Random
using Plots
using LoopVectorization

In [15]:
# initialize SPD power manifold P(d)^d1
dd = 3  # size of the SPD matrices, i.e., ∈ R^d×d
M_base = SymmetricPositiveDefinite(dd)
d = manifold_dimension(M_base)
d1 = 100  # size of the signal
M = PowerManifold(M_base, NestedPowerRepresentation(), d1)

PowerManifold(SymmetricPositiveDefinite(3), NestedPowerRepresentation(), 100)

### Generate a data set

In [16]:
e = 1. * Matrix(I, dd, dd)
# compute basis on M_base
Θ = get_basis(M_base, e, DefaultOrthonormalBasis())
#  construct data
Q = fill(e, d1)
# draw random tvectors 
τ₁ = 4.  # variance
τ₂ = 3.  # variance
σ = .05  # variance
# Xₑⁱ = zeros(d)
# Xₑⁱ[1] = 1.
# Xₑ = get_vector(M_base, e, Xₑⁱ, Θ)
Xₑ = Θ.data[1]
Yₑ = Θ.data[6]

Random.seed!(31)
predata = [exp(M_base, e, sqrt(τ₁) * randn(1)[1] * Xₑ + sqrt(τ₂) * randn(1)[1] * Yₑ) for i in 1:d1]

data = [exp(M_base, predata[i], random_tangent(M_base, predata[i], Val(:Gaussian), σ)) for i in 1:d1] # ∈ P(3)^d1
log_Q_data = log(M, Q, data)  # ∈ T_e P(3)^d1
println(Xₑ)

[1.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]


We want to investigate how reliable both ranks are, i.e., we don't want that something looks low-rank, but in reality is higher rank

In [17]:
# compute SVD
# compute Gramm Matrix
Gramm_Q = [inner(M_base, e, log_Q_data[k], log_Q_data[l]) for k=1:d1, l=1:d1]

# compute Σ and V
(sqSigma, V) = eigen(Symmetric(Gramm_Q), d1-min(d1-1,d-1):d1)
# compute U
U = [1/sqrt(sqSigma[i]) * sum([V[k,i] * log_Q_data[k] for k in 1:d1]) for i in 1:d]

sqSigma

6-element Vector{Float64}:
   0.29045302658596506
   0.5218338723531915
   0.798002490790501
 131.06330076224873
 254.36833818532568
 572.1462171349498

In [18]:
# Gramm_Q = zeros(d1,d1)
# # @avx for k in 1:d1
# #     for l in 1:d1
# #         Gramm_Q[k,l] = inner(M_base, e, log_Q_data[k], log_Q_data[l])
# #     end
# # end

# @avx for k in 1:d1, l in 1:d1
#     Gramm_Q[k,l] = inner(M_base, e, log_Q_data[k], log_Q_data[l])
# end

In [19]:
U[end]

3×3 Matrix{Float64}:
  0.959775   -0.080951   -0.0545426
 -0.080951    0.0801387   0.0588617
 -0.0545426   0.0588617  -0.215464

In [20]:
function β(κ)
    (κ < 0) && return sinh(sqrt(-κ)) / ( sqrt((-κ)))
    (κ > 0) && return sin(sqrt(κ)) / (sqrt(κ))
    return 1.0 # cuvature zero.
end

β (generic function with 1 method)

In [21]:
# we want to compute the metric tensor entries in default ONB at e and then compute g_ij so that we don't have to reevaluate this all the time
gᵢⱼ = zeros(d,d)
for k=1:d1
    Ξₖ = get_basis(M_base, e, DiagonalizingOrthonormalBasis(log_Q_data[k]))
    κₖ = Ξₖ.data.eigenvalues
    βₖ = [β(κₖ[l]) for l in 1:d]
    gᵢⱼ += 1/d1 * [sum([βₖ[l]^2 * inner(M_base, e, Θ.data[i], Ξₖ.data.vectors[l]) * inner(M_base, e, Θ.data[j], Ξₖ.data.vectors[l]) for l=1:d]) for i=1:d, j=1:d]
end
gᵢⱼ

6×6 Matrix{Float64}:
  22.5431   -12.783    16.1928   -1.80095  -23.7938  -19.7421
 -12.783     29.7694  -43.4081  -25.8441    33.9666   38.627
  16.1928   -43.4081   71.9744   43.0299   -50.6284  -59.2227
  -1.80095  -25.8441   43.0299   36.5166   -24.0955  -33.7156
 -23.7938    33.9666  -50.6284  -24.0955    46.7982   47.8893
 -19.7421    38.627   -59.2227  -33.7156    47.8893   54.4578

In [22]:
# compute SVD
# compute rescaled Gramm Matrix
reweighted_Gramm_Q = [sum([gᵢⱼ[i,j] * get_coordinates(M_base, e, log_Q_data[k], Θ)[i] * get_coordinates(M_base, e, log_Q_data[l], Θ)[j] for i=1:d, j=1:d]) for k=1:d1, l=1:d1]
# compute Σ and V
(reweighted_sqSigma, reweighted_V) = eigen(Symmetric(reweighted_Gramm_Q ), d1-min(d1-1,d-1):d1)
# compute U
reweighted_U = [1/sqrt(reweighted_sqSigma[i]) * sum([reweighted_V[k,i] * log_Q_data[k] for k in 1:d1]) for i in 1:d]

reweighted_sqSigma

6-element Vector{Float64}:
     0.553159059049444
     1.174033631000456
     4.333009298457721
   329.4610718107315
  4400.957045182701
 24138.794902397054

In [23]:
reweighted_U[4]

3×3 Matrix{Float64}:
  0.332818  -0.304019  -0.208999
 -0.304019   0.300906   0.216074
 -0.208999   0.216074   0.0213718

In [24]:
ref_distance = distance(M, data, Q)
tangent_distances_r = zeros(d)
distances_r = zeros(d)
reweighted_tangent_distances_r = zeros(d)
reweighted_distances_r = zeros(d)

for rank in 1:d
    log_Q_data_r = [sum([U[i] * sqrt(sqSigma[i]) * V[k,i] for i in d-rank+1:d]) for k in 1:d1]
    reweighted_log_Q_data_r = [sum([reweighted_U[i] * sqrt(reweighted_sqSigma[i]) * reweighted_V[k,i] for i in d-rank+1:d]) for k in 1:d1]
    # expoentiate back
    data_r = exp(M, Q, log_Q_data_r)
    reweighted_data_r = exp(M, Q, reweighted_log_Q_data_r)


    # compute tangent space error
    tangent_distances_r[rank] = norm(M, Q,  log_Q_data - log_Q_data_r)
    reweighted_tangent_distances_r[rank] = norm(M,Q,  log_Q_data - reweighted_log_Q_data_r )


    # compute manifold error
    distances_r[rank] = distance(M, data, data_r)
    reweighted_distances_r[rank] = distance(M, data, reweighted_data_r)
end
# TODO plot distances for every rank 1:6
# It seems to pick the correct vectors, but the singular values do not yet reflect that well how low the rank actually is...
# -> we can reweight the U's and the sigmas by normalizing them wrt old metric
# the SPD manifold always has a couple of directions with negative curvature, check whether these are the same as the obtained directions

In [25]:
plot(1:d, [ref_distance * ones(d), distances_r, reweighted_distances_r], label = ["reference" "standard inner product" "reweighted inner product"])
savefig("P3_example2.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/P3_example2.png"